In [299]:
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import xml.etree.ElementTree as ET
import tensorflow as tf
from argparse import Namespace
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anany\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anany\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [307]:
# This function is to remove stopwords to create relation in KG
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return filtered_sentence

In [116]:
#Parse XML
tree = ET.parse('medicine_kg.xml')
root = tree.getroot()
medicine_kg = []
synonym_ref = {}
for i in range(len(root)):
    drug_name = root[i].find("{http://www.drugbank.ca}name").text
    
    synonym_ref[drug_name] = []
    # This is needed for comparing the names in prescription.csv with drugbank id to form patient medicine bipartite graph
    synonym_element = root[i].find("{http://www.drugbank.ca}synonyms")
    for synonym in synonym_element:
        if synonym.text is not None: 
            synonym_ref[drug_name].append(synonym.text.lower())
    if len(synonym_ref[drug_name]) == 0:
        synonym_ref[drug_name].append(drug_name.lower())
    
    # This is needed for finding drug to drug interaction and constructing the medicine knowledge graph 
    interaction_element = root[i].find("{http://www.drugbank.ca}drug-interactions")
    
    
    for interaction in interaction_element:
        interaction_name = interaction[1].text
        description = interaction[2].text
        replace_arr = ["may", "combined", ".","risk", "severity", ",", 'used', 'combination'] + drug_name.lower().split(" ") + interaction_name.lower().split(" ")
        
        description_arr = remove_stopwords(description)
        cleaned_description = [w for w in description_arr if not w.lower() in replace_arr]
        find_adj = ["increases", "increased", "increase", "decreases", "decreased", "decrease"]
        found_adj = [x for x in cleaned_description if x in find_adj]
        
        if len(found_adj) > 0:
            cleaned_description.remove(found_adj[0])
            if found_adj[0].find('increase') != -1:
                cleaned_description.insert(0, 'increases')
            else:
                cleaned_description.insert(0, 'decreases')
        relation = {
            'source': drug_name,
            'edge': ' '.join(cleaned_description),
            'target': interaction_name
            
        }
        medicine_kg.append(relation)

In [143]:
# Splitting medicine knowledge graph triplets to train, test and valid to calculate embedding by TransD
medicine_kg_df = pd.DataFrame.from_records(medicine_kg)
X_train, X_test_val = train_test_split(medicine_kg_df, test_size=0.6)
X_test, X_val = train_test_split(X_test_val, test_size=0.5)
X_train.to_csv('./med-train.csv',sep='\t')
X_val.to_csv('./med-valid.csv',sep='\t')
X_test.to_csv('./med-test.csv',sep='\t')

In [144]:
# MIMIC iii prescription has patient to drug
prescription_df = pd.read_csv('PRESCRIPTIONS.csv')
prescription_df = prescription_df[['subject_id', 'drug', 'dose_val_rx', 'dose_unit_rx', 'form_unit_disp']]
prescription_df = prescription_df[(prescription_df['form_unit_disp'] != 'SYR') & (prescription_df['form_unit_disp'] != 'BAG') & (prescription_df['form_unit_disp'] != 'CAN')]
prescription_df = prescription_df[prescription_df['drug'].notna()]
prescription_df['drug'] = prescription_df['drug'].str.lower()

In [145]:
# Since the drugs are not in standard form, look in synonym from drugbank
drugs = prescription_df['drug'].values
drugs = list(set(drugs))
final_drugs = {}
synonyms = list(synonym_ref.values())
actual_drug = list(synonym_ref.keys())
for i in drugs:
    for j in range(0, len(synonyms)):
        if i in synonyms[j]:
            final_drugs[i] = actual_drug[j]

In [146]:
col1 = list(final_drugs.keys())
col2 = list(final_drugs.values())
data = {'drug': col1, 'actual_drug': col2}
drug_df = pd.DataFrame.from_dict(data)
drug_df = drug_df[drug_df['drug'].notna()]

In [147]:
# Replace the drug name with the drugname in drugbank to connect to medicine knowledge graph
final_df = pd.merge(prescription_df, drug_df, how='left', left_on='drug', right_on='drug')
final_df = final_df[final_df['actual_drug'].notna()]
final_df.drop('drug', 1, inplace=True)
final_df.rename(columns={'actual_drug': 'drug'}, inplace=True)

In [148]:
# Create count column with number of times the drug prescribed to the patient
pm_df = final_df.groupby(['subject_id', 'drug']).size().to_frame('count').reset_index()

In [149]:
# Construct bipartite graph
G = nx.Graph()
G.add_nodes_from(pm_df['subject_id'], bipartite=0)
G.add_nodes_from(pm_df['drug'], bipartite=1)
G.add_weighted_edges_from(
    [(row['subject_id'], row['drug'], row['count']) for idx, row in pm_df.iterrows()], weight='weight')

In [150]:
class DataLoader:
    def __init__(self,G):
        self.g=G
        self.num_of_nodes=self.g.number_of_nodes()
        self.num_of_edges=self.g.number_of_edges()
        self.edges_raw=list(self.g.edges(data=True))
        self.nodes_raw=list(self.g.nodes())
        self.edge_distribution=np.array([attr['weight'] for _,_,attr in self.edges_raw],dtype=np.float32)
        self.edge_distribution/=np.sum(self.edge_distribution)
        self.node_negative_distribution=np.power(np.array([self.g.degree(node) for node in self.nodes_raw],dtype=np.float32),0.75)
        self.node_negative_distribution/=np.sum(self.node_negative_distribution)
        self.node2idx={}
        self.idx2node={}
        for idx,node in enumerate(self.nodes_raw):
            self.node2idx[node]=idx
            self.idx2node[idx]=node
        self.edges=[(self.node2idx[u],self.node2idx[v]) for u,v,_ in self.edges_raw]
        
    def fetch_batch(self,batch_size=5,K=3):
        edge_batch_idx=np.random.choice(self.num_of_edges,size=batch_size,p=self.edge_distribution)
        u_i=[]
        u_j=[]
        label=[]
        for edge_idx in edge_batch_idx:
            edge=self.edges[edge_idx]
            if np.random.rand()>0.5:
                edge=(edge[1],edge[0])
            u_i.append(edge[0])
            u_j.append(edge[1])
            label.append(1)
            for i in range(K):
                while True:
                    negative_node=np.random.choice(self.num_of_nodes,p=self.node_negative_distribution)
                    if not self.g.has_edge(self.idx2node[edge[0]],self.idx2node[negative_node]):
                        break
                u_i.append(edge[0])
                u_j.append(negative_node)
                label.append(-1)
        return u_i,u_j,label
        
    def embedding_mapping(self,embedding):
        return {node:embedding[self.node2idx[node]] for node in self.nodes_raw}

In [151]:
class LINE:
    def __init__(self,args):
        tf.compat.v1.disable_eager_execution()
        self.u_i=tf.compat.v1.placeholder(dtype=tf.int32,shape=[args.batch_size*(args.K+1)],name='u_i')
        self.u_j=tf.compat.v1.placeholder(dtype=tf.int32,shape=[args.batch_size*(args.K+1)],name='u_j')
        self.label=tf.compat.v1.placeholder(dtype=tf.float32,shape=[args.batch_size*(args.K+1)],name='label')
        with tf.compat.v1.variable_scope('embedding',reuse=True):
            self.embedding=tf.Variable(tf.compat.v1.truncated_normal([args.num_of_nodes,args.embedding_dim]))
        self.u_i_embedding=tf.nn.embedding_lookup(self.embedding,self.u_i)
        if args.proximity=='first-order':
            self.u_j_embedding=tf.nn.embedding_lookup(self.embedding,self.u_j)
        elif args.proximity=='second-order':
            with tf.compat.v1.variable_scope('context_embedding',reuse=True):
                self.context_embedding=tf.Variable(tf.compat.v1.truncated_normal([args.num_of_nodes,args.embedding_dim]))
            self.u_j_embedding=tf.nn.embedding_lookup(self.context_embedding,self.u_j)
        self.inner_product=tf.reduce_sum(self.u_i_embedding*self.u_j_embedding,axis=1)
        self.loss=-tf.reduce_mean(tf.compat.v1.log_sigmoid(self.label*self.inner_product))
        self.learning_rate=tf.compat.v1.placeholder(dtype=tf.float32,name='learning_rate')
        self.optimizer=tf.compat.v1.train.RMSPropOptimizer(learning_rate=self.learning_rate)
        self.train_op=self.optimizer.minimize(self.loss)

In [152]:
# Calculating LINE embedding for the bipartite graph
def train(graph):
    args=Namespace()
    args.proximity='second-order'
    args.embedding_dim=50
    args.num_batches=100
    args.K=3
    args.batch_size=5
    args.learning_rate=0.001
    
    data_loader=DataLoader(graph)
    args.num_of_nodes=data_loader.num_of_nodes
    model=LINE(args)
    with tf.compat.v1.Session() as sess:
        tf.compat.v1.global_variables_initializer().run()
        initial_embedding=sess.run(model.embedding)
        learning_rate=args.learning_rate
        for i in range(args.num_batches):
            u_i,u_j,label=data_loader.fetch_batch(batch_size=args.batch_size,K=args.K)
            feed_dict={model.u_i:u_i,model.u_j:u_j,model.label:label,model.learning_rate:learning_rate}
            loss,_=sess.run([model.loss,model.train_op],feed_dict=feed_dict)
        if i==args.num_batches-1:
            embedding=sess.run(model.embedding)
            normalized_embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)
            return data_loader.embedding_mapping(normalized_embedding)

In [153]:
#Patient-medicine embedding dict
embedding_dict = train(G)

In [154]:
keys = embedding_dict.keys()
patient_embedding = {}
medicine_embedding = {}
for i in list(keys):
    if str(i).isnumeric():
        patient_embedding[str(i)] = embedding_dict[i]
    else:
        medicine_embedding[i] = embedding_dict[i]

In [163]:
# TransD embedding for the drugs of medicine knowledge graph
medicine_data = pd.read_csv ("./pykg2vec/examples/data/embeddings/transd/ent_embedding.tsv", sep = '\t', header=None)
label_data = pd.read_csv ("./pykg2vec/examples/data/embeddings/transd/ent_labels.tsv", sep = '\t', header=None)
medicine_data = medicine_data.to_numpy()
label_data = label_data.to_numpy()
medicine_kg_embedding = {}
for index, label in enumerate(label_data):
    medicine_kg_embedding[label[0]] = medicine_data[index]
    
# TransD embedding for the relations in medicine knowledge graph    
med_rel_data = pd.read_csv ("./pykg2vec/examples/data/embeddings/transd/rel_embedding.tsv", sep = '\t', header=None)
med_label_data = pd.read_csv ("./pykg2vec/examples/data/embeddings/transd/rel_labels.tsv", sep = '\t', header=None, encoding='unicode_escape')
med_rel_data = med_rel_data.to_numpy()
med_label_data = med_label_data.to_numpy()
med_rel_embedding = {}
for index, label in enumerate(med_label_data):
    med_rel_embedding[label[0]] = med_rel_data[index]

In [306]:
# To construct disease knowledge graph, use the ICD9 Ontology dataset
disease_df = pd.read_csv('ICD9CM.csv')
disease_df = disease_df[['Class ID','Parents']]
disease_df['type'] = 'childOf'
disease_df = disease_df.rename(columns={"Class ID": "target", "Parents": "source"})
disease_df = disease_df[['target','type','source']]
disease_df['source'] = disease_df.source.apply(lambda x: str(x).split('/')[-1].replace('.', ''))
disease_df['target'] = disease_df.target.apply(lambda x: str(x).split('/')[-1].replace('.', ''))
# Split into train, test and valid to calculate the embedding
X_train, X_test_val = train_test_split(disease_df, test_size=0.6)
X_test, X_val = train_test_split(X_test_val, test_size=0.5)
X_train.to_csv('./disease-train.csv',sep='\t')
X_val.to_csv('./disease-valid.csv',sep='\t')
X_test.to_csv('./disease-test.csv',sep='\t')

In [157]:
# Construct patient disease bipartite network
disease_patient_df = pd.read_csv('DIAGNOSES_ICD.csv')
disease_patient_df = disease_patient_df[['subject_id','icd9_code']]
G1 = nx.Graph()
G1.add_nodes_from(disease_patient_df['subject_id'], bipartite=0)
G1.add_nodes_from(disease_patient_df['icd9_code'], bipartite=1)
G1.add_weighted_edges_from(
    [(row['subject_id'], row['icd9_code'], 1) for idx, row in disease_patient_df.iterrows()], weight='weight')

In [158]:
disease_embedding_dict = train(G1)
keys = disease_embedding_dict.keys()
disease_embedding = {}

for i in list(keys):
    if str(i).isnumeric():
        lists = []
        if str(i) in patient_embedding:
            lists.append(patient_embedding[str(i)])
        lists.append(disease_embedding_dict[i])
        # Combine the patient embedding from p-m bipartite network and p-d bipartite network
        patient_embedding[str(i)] = list(map(sum, zip(*lists)))
    else:
        disease_embedding[i] = disease_embedding_dict[i]

In [308]:
# Combine the medicine embedding from p-m bipartite network and medicine knowledge graph
for i in medicine_kg_embedding:
    if i in medicine_embedding:
        lists = []
        lists.append(medicine_embedding[i])
        lists.append(medicine_kg_embedding[i])
        medicine_kg_embedding[i] = list(map(sum, zip(*lists)))

In [160]:
# Get the disease embedding from disease knowledge graph and combine with embedding from p-d knowledge graph
disease_data = pd.read_csv ("./pykg2vec/examples/disease-kg/embeddings/transd/ent_embedding.tsv", sep = '\t', header=None)
disease_label_data = pd.read_csv ("./pykg2vec/examples/disease-kg/embeddings/transd/ent_labels.tsv", sep = '\t', header=None)
disease_data = disease_data.to_numpy()
disease_label_data = disease_label_data.to_numpy()
medicine_kg_embedding = {}
for index, label in enumerate(disease_label_data):
    lists = []
    if label[0] in disease_embedding:
        lists.append(disease_embedding[label[0]])
    lists.append(disease_data[index])
    disease_embedding[label[0]] = list(map(sum, zip(*lists)))

In [279]:
p = 10019

In [290]:
sepsis_drug = ['Vancomycin','Ceftriaxone','Cefuroxime','Ceftin','Vancocin']
hypertension_drug = ['Lisinopril','Amlodipine','Norvasc','Carvedilol','Furosemide']
resp_failure_drug = ['Acetaminophen','Norepinephrine','Prednisone','Albuterol','Noxivent']

In [281]:
patient_em = patient_embedding[str(p)]

In [297]:
def dot_product(arr1, arr2):
    prod = 0
    for i, v in enumerate(arr1):
        prod = prod + (arr1[i] * arr2[i])
    return prod

def l1_norm(arr):
    norm = 0
    for i in arr:
        norm = norm + abs(i)
    return norm

def sum_arr(arr1, arr2):
    return_arr = []
    for i, v in enumerate(arr1):
        return_arr.append(arr1[i] + arr2[i])
    return return_arr

def diff_arr(arr1, arr2):
    return_arr = []
    for i, v in enumerate(arr1):
        return_arr.append(arr1[i] - arr2[i])
    return return_arr

def scalar_product(scalar, arr):
    return_arr = []
    for i in arr:
        return_arr.append(i * scalar)
    return return_arr

In [311]:
def calculate_ddi(patient_em, arr1, arr2):
    n = 5
    res_scores = {}
    
    for index, value in enumerate(arr1):
        if value in list(medicine_kg_embedding.keys()): 
            res = dot_product(patient_em, medicine_kg_embedding[value])
            scores[value] = res
            
    for i in range(0, n):
        value = arr1[i]
        if value in scores:
            for j in arr2:
                ddi = 0
                relation_df = medicine_kg_df[(medicine_kg_df['source'] == value) & (medicine_kg_df['target'] == j)]
                relation = relation_df['edge'].unique()
                if len(relation) > 0:
                    relation = relation[0]
                    ddi = ddi + l1_norm(diff_arr(sum_arr(medicine_kg_embedding[value], med_rel_embedding[relation]), medicine_kg_embedding[j]))
            res_scores[value] = scores[value] - ddi
    return res_scores

In [293]:
sepsis_ddi = calculate_ddi(patient_em, sepsis_drug, hypertension_drug)
sepsis_ddi

{'Vancomycin': -554.9162230729705,
 'Ceftriaxone': -557.8339226240262,
 'Cefuroxime': -558.7119698473205}

In [294]:
hypertension_ddi = calculate_ddi(patient_em, hypertension_drug, sepsis_drug)
hypertension_ddi

{'Lisinopril': -14.056283833129838,
 'Amlodipine': 3.3913091023758932,
 'Carvedilol': -4.981197779652382,
 'Furosemide': 12.222794158188186}

In [296]:
resp_failure_ddi = calculate_ddi(patient_em, resp_failure_drug, sepsis_drug)
resp_failure_ddi

{'Acetaminophen': 34.319806024994,
 'Norepinephrine': -4.349440260140723,
 'Prednisone': -6.313434044598619}

In [309]:
# 99592 - sepsis
# 4019 - unspecified essential hypertension

diseases = ['99592', '4019']
for i, val in enumerate(diseases):
    d = disease_embedding[val]
    res_scalar = scalar_product(math.exp(-(i+1)), d)
    if i == 0:
        new_patient_embedding = res_scalar
    else:
        new_patient_embedding = sum_arr(new_patient_embedding, res_scalar)
print(new_patient_embedding)

[0.22713131428177216, -0.06436416200443511, -0.06797525378304467, 0.11078386062666923, 0.08790227845927051, 0.030545909622186413, 0.1489080885635613, -0.09045192235260764, -0.12998357313164038, -0.06548483085180652, -0.14461861927186187, 0.04129065622747197, -0.0033810655376894025, -0.06675221620062702, -0.08474981671774566, -0.017934030150872325, -0.05666303840875047, 0.12161384445041852, 0.059725237774324244, 0.30432979887447953, 0.09906956352084226, -0.23363533089997313, -0.002737359841358377, -0.10742571750330182, -0.12811973155023276, 0.13554820451096083, -0.007797355628315655, -0.08919129111385406, 0.09034248716057855, -0.04636084356218427, 0.10136227174155837, 0.018091533589574334, 0.047255239550534844, -0.06489889817160005, 0.027954206776750957, -0.013842769886809839, 0.32765129684885635, -0.055574395194754406, -0.2457427589361605, -0.05581639173345987, -0.07683065207616775, -0.19231008188434578, 0.355714113452659, 0.3364620643825719, 0.11005410031536546, 0.06878400394254308, 0

In [312]:
sepsis_ddi = calculate_ddi(new_patient_embedding, sepsis_drug, hypertension_drug)
sepsis_ddi

{'Vancomycin': -548.910495202641,
 'Ceftriaxone': -545.8999233213062,
 'Cefuroxime': -561.4798174361738}